# **EDA defectos**

In [1]:
%%HTML
<script src = "require.js"></script>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import pandas as pd
import seaborn as sns

# scipy stuff 
import scipy.signal as signal
import scipy.stats as stats
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
from fitter import Fitter, get_common_distributions, get_distributions

import plotly.graph_objects as go

pio.renderers.default = "notebook"

# Librerias propias
import sys
sys.path.append('../Librerias')
import dataset as ds
import similaritymeasures


sns.set_style("darkgrid")

ModuleNotFoundError: No module named 'dataset'

In [ ]:

fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/"
#fpath = "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/"
fname = "datosconsu_021023_bajos.mat"
cutoff = [8/1000, 11/1000]

set =  ds.MatFileToDataFrame(fpath, fname)
df = set.get_dataframe(cutoff)
df.tail()

In [ ]:
amplitude_env = np.abs(df['Hilbert Transform'])
inst_phase =  np.unwrap(np.angle(df['Hilbert Transform']))
inst_freq = (np.diff(inst_phase)/(2.0*np.pi)*1000)
diff_phase =  np.diff(inst_phase)
mean_phase = np.mean(diff_phase)
diff_phase = np.insert(diff_phase, 0, 0)  

In [ ]:
df['Amplitude Envelope'] = amplitude_env
df['Instantaneous Phase'] = inst_phase

In [ ]:
# filtro gradiente de fase

cutoff = 0.1
order = 4

b,a = signal.butter(order, cutoff, btype='low')
grad_phase = signal.filtfilt(b, a, diff_phase-mean_phase)

df['Gradient Phase'] = grad_phase

In [ ]:
df.head()

### **Señal Original**

Se buscan las frecuencias características de la señal. Se sabe que la fuente vibra a 20Hz y que la frecuencia más predominante será su sub-armonico a 10 Hz. Buscamos corroborar esto y ver si hay otras frecuencias interesantes.

In [ ]:
mean_signal = df['Signal - Mean'].values
sig_fft = fft(mean_signal)
N = len(mean_signal)
sampling_rate = 1000.0
normalize =  N/2.0

freq_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(sig_fft)/normalize

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = freq_axis, y = norm_amplitude, mode = 'lines'))
fig.update_layout(title = 'Espectro de amplitud', xaxis_title = 'Frecuencia [Hz]', yaxis_title = 'Amplitud')
fig.update_xaxes(range=[0, 50])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_axis, y=norm_amplitude, mode='lines'))
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.update_layout(title='Signal Spectrum', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()


Se ve claramente el sub-armonico seguido por el peak del armonico. Por lo que se puede decir que la señal tiene dos frecuencias características que corresponden 
a 0.1s y 0.05s respectivamente, en periodos, esto implica que la señal principal se presenta cada 100 muestras y la secundaria cada 50.

## **Señal Filtrada**

Para hacer los calculos de Hilbert, se filtra la señal con un cutoff especificado al momento de generar el dataframe. En este caso es de [8Hz, 11Hz]. Esto se hace para eliminar el ruido de alta frecuencia que no nos interesa. Por lo mismo, en esta señal se debería encontrar el subarmonico que corresponde a los 10Hz como componente principal.


In [ ]:
filtered_signal = df['Filtered Signal'].values

sig_fft = np.fft.fft(filtered_signal)
N = len(filtered_signal)
normalize = N/2.0

freq_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(sig_fft)/normalize

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = freq_axis, y = norm_amplitude, mode = 'lines'))
fig.update_layout(title = 'Espectro de amplitud', xaxis_title = 'Frecuencia [Hz]', yaxis_title = 'Amplitud')
fig.update_xaxes(range=[0, 15])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_axis, y=norm_amplitude, mode='lines'))
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.update_layout(title='Signal Spectrum', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()

Se ve el peak de la onda de faraday  alos 10Hz y se puede ver que existe ruido a una frecuencia cercana a los 4.5Hz.

## **Onda Envolvente**

In [ ]:
print(f'Mínima amplitude envolventes: \t{np.min(np.abs(df["Amplitude Envelope"]))}')
print(f'Máxima amplitude envolventes: \t{np.max(np.abs(df["Amplitude Envelope"]))}')
print(f'STD amplitude envolventes: \t{np.std(np.abs(df["Amplitude Envelope"]))} \n')

print(f'Mínima gradiente de fase: \t{np.min(df["Gradient Phase"])}')
print(f'Máxima gradiente de fase: \t{np.max(df["Gradient Phase"])}')
print(f'STD gradiente de fase: \t{np.std(df["Gradient Phase"])} \n')

In [ ]:
# Fourier Transform for AE
fourier = fft(df['Amplitude Envelope'].values)
N = len(df['Amplitude Envelope'])
sampling_rate = 1000.0
normalize = N/2.0

frequency_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(fourier)/normalize

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_axis, y=norm_amplitude, mode='lines'))
#fig.update_xaxes(range=[-2, 2])
fig.update_layout(title='Close Up spectrum for AE', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()

In [ ]:
# loglog plot for AE spectrum

fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_axis, y=norm_amplitude, mode='lines'))
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.update_layout(title='Close Up spectrum for AE', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()


Peaks de frecuencia se encuentran en los 10Hz, que es residuo de Faraday. Presenta otro peak cerca de los 4.8Hz, residual del ruido no eliminado de Faraday y componentes entre los 0.01 y 0.1Hz.

In [ ]:
#filtro pasa baja de 4Hz
cutoff_amp = 7/1000
a, b = signal.butter(4, cutoff_amp , btype='high')
c, d = signal.butter(4, cutoff_amp, btype = 'low')
amp_filtered =  signal.filtfilt(a, b, df['Amplitude Envelope'].values)
amp_filtered2 = signal.filtfilt(c, d, df['Amplitude Envelope'].values)

In [ ]:
fourier = fft(amp_filtered)
N = len(amp_filtered)
sampling_rate = 1000.0
normalize = N/2.0

frequency_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(fourier)/normalize

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_axis, y=norm_amplitude, mode='lines'))
#fig.update_xaxes(range=[-1, 1])
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.update_layout(title='Espectro AE filtrado', xaxis_title='Frequency [Hz]', yaxis_title='Amplitude')
fig.show()

En caso de estos puntos ser significativos, indican que la señal presenta un componente con frecuencias entre 0.001 a 0.1Hz que corresponden a periodos de 1000 a 10 segundos. Esto implica que tienen ciclos cada $10^6$ y $10^4$ muestras respectivamente.


### **amplitud mínima**

In [ ]:
min_amp = np.min(np.abs(df["Amplitude Envelope"]))

#30 es un valor arbitrario dado que esa es la cantidad de señales que sé que puedo reconocer a simple vista
indices = df[df['Amplitude Envelope'] < min_amp*30].index
len(indices)

In [ ]:
#función para eliminar los indices consecutivos de un arreglo
def delete_consecutivos(array):
    output = []
    for i in range(len(array)):
        if i == 0:
            output.append(array[i])
        else:
            if array[i] != array[i-1]+1:
                output.append(array[i])
    return output


In [ ]:
#indices = delete_consecutivos(indices)
len(indices)

## **peaks en gradiente fase**

In [ ]:
peaks, peak_info = signal.find_peaks(np.abs(diff_phase-mean_phase), [0, 5.0], threshold = 0, width = [0, 100] , prominence = 0,
                                     plateau_size=None, distance=100)

print(f'Nº de peaks: \t {len(peaks)}')
print(f'Amplitud promedio: \t {np.mean(peak_info["peak_heights"])}')
print(f'Amplitud máxima: \t {np.max(peak_info["peak_heights"])}')
print(f'Amplitud mínima: \t {np.min(peak_info["peak_heights"])}')
print(f'Amplitud STD: \t {np.std(peak_info["peak_heights"])}')

notar que la desviación estándar es muy grande, es cercano al 200 veces la amplitud mínima. Por lo que no es un buen indicador en como estan repartidos los peaks.

Argumentos:

* **Threshold** de 0, es la distancia vertical a al vecino vertical más cercano. Se elije este número porque es el mínimo posible y con esto, se puede acceder a la información que da esta opción.
* **Width** de 0 a 100 es el ancho de la ventana de búsqueda de picos. Se elijen estos números por la frecuencia de sampleo.
* **Distance** de 0 a 100 es la distancia mínima entre picos. Se elijen estos números por la frecuencia de sampleo.
* **Prominence** de 0, prominencia de pico. Es el valor mínimo posible y se elije para acceder a la información que da esta opción.
* **Height** de 0.01 a 5.0, altura en la que se encuentran los picos. La gran mayoría de los defectos reconocibles a simplevista tienen alturas superiores a 0.01.
* **Plateau size** no se utiliza, dado que no se buscan plataformas en la señal.

In [ ]:
peak_info.keys()

### **intersección amplitud y peaks**

In [ ]:
intersection = np.intersect1d(indices, peaks, assume_unique=False, return_indices=False)
len(intersection)

In [ ]:
for k, i in enumerate(intersection[:3]):
   fig =  go.Figure()
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = amp_filtered[i-500:i+500], name='AE Filtrada'))#, line_shape='linear'))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = np.abs(df['Gradient Phase'][i-500:i+500]), name='Instantaneous Phase'))#, line_shape = 'linear'))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = (df['Filtered Signal'][i-500:i+500]), name='Filtered Signal'))#, line_shape = 'linear'))']))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = df['Amplitude Envelope'][i-500:i+500], name='AE'))#, line_shape='linear'))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = amp_filtered2[i-500:i+500], name='AE Filtrada2'))
   fig.update_layout(title = f'Peak indice {i}, curva {k}')
   fig.show()


Componente de 10Hz genera los pasos por cero. Las frecuencias menores tienen efectos en la amplitud de la envolvente y que esta vaya oscilando lejos del origen.

In [ ]:
for k, i in enumerate(intersection):
   fig =  go.Figure()
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = df['Amplitude Envelope'][i-500:i+500], name='Amplitude Envelope'))#, line_shape='linear'))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = np.abs(df['Gradient Phase'][i-500:i+500]), name='Instantaneous Phase'))#, line_shape = 'linear'))
   fig.add_trace(go.Scatter(x = df.index[i-500:i+500], y = (df['Filtered Signal'][i-500:i+500]), name='Filtered Signal'))#, line_shape = 'linear'))']))
   fig.update_layout(title = f'Peak indice {i}, curva {k}')
   fig.show()

#### **prominence**

In [ ]:
# busco prominences de los peaks que salieron de la intersección
info_prominence = signal.peak_prominences(np.abs(diff_phase-mean_phase), intersection, wlen=None)[0]
contour_heights = np.abs(info_prominence - df['Gradient Phase'][intersection])


In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(df.index, np.abs(df['Gradient Phase']), label='Gradient Phase')
plt.scatter(intersection, np.abs(df['Gradient Phase'][intersection]), marker='x', label='defectos', color = 'violet', alpha = 0.5)
#plt.plot(df.index, df['Amplitude Envelope'], label='Amplitude Envelope')
plt.vlines(intersection, ymin = contour_heights, ymax = df['Gradient Phase'][intersection], label='Contour Heights', color = 'grey', linestyles='dashed', alpha = 0.5)
plt.title('Prominences')
plt.legend()
plt.show()



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Gradient Phase'], name='Gradient Phase'))
fig.add_trace(go.Scatter(x=intersection, y=df['Gradient Phase'][intersection], mode='markers', name = 'defectos'))
fig.add_trace(go.Scatter(x = df.index, y = df['Amplitude Envelope'], name = 'Amplitude Envelope'))
fig.add_trace(go.Scatter(x=intersection, y=contour_heights, mode='markers', name='Contour Heights'))
fig.update_layout(title='Prominences')
fig.show()



#### **width**

In [ ]:
widths = signal.peak_widths(np.abs(diff_phase-mean_phase), intersection, rel_height=0.5)

print(f'Ancho promedio: \t {np.mean(widths[0])}')
print(f'Media anchos: \t {np.median(widths[0])}')
print(f'Ancho máximo: \t {np.max(widths[0])}')
print(f'Ancho mínimo: \t {np.min(widths[0])}')
print(f'Ancho STD: \t {np.std(widths[0])}')


In [ ]:
min_width_index = np.argmin(widths[0])
max_width_index = np.argmax(widths[0])

min_width_peak = intersection[min_width_index]
max_width_peak = intersection[max_width_index]

print("Peak with minimum width:", min_width_peak)
print("Peak with maximum width:", max_width_peak)


## **ventanas con peaks**

In [ ]:
defectos = []
for i in intersection:
    defectos.append(df.iloc[i-100:i+100])

defectos[-1].tail()

In [ ]:
anchos = []
for elem in defectos:

    subset = np.abs(elem['Gradient Phase'].values)
    min1 = np.min(subset[:25])
    index1 = np.where(subset[:25] == min1)[0][0]

    min2 =  np.min(subset[25:])
    index2 = np.where(subset[25:] == min2)[0][0]
    index2 = index2+24
    anchos.append([index1, index2])

In [ ]:
for i, elem in enumerate(defectos):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=elem.index, y=elem['Amplitude Envelope'], name='Amplitude Envelope'))
    fig.add_trace(go.Scatter(x=elem.index, y=(elem['Gradient Phase']), name='Gradient Phase'))
    fig.add_trace(go.Scatter(x=elem.index, y=elem['Filtered Signal'], name='Filtered Signal'))
    fig.update_layout(title=f'defecto {i}')
    fig.show()

#### **tiempos**


Se busca de manera manual las amplitudes de los defectos y con esto se sacan estadisticas de los mismos.

In [ ]:
manual_width = [
    [25014, 25042],
    [27460, 27487],
    [35630, 35658],
    [37207, 37236],
    [48343, 48372],
    [57421, 57450],
    [61722, 61750],
    [78988, 79016],
    [82306, 82336],
    [84344, 84373],
    [97845, 97875], #10
    [99883, 99914], 
    [101083, 101111],
    [131884, 131912],
    [165590, 165619],
    [166234, 166264],
    [183081, 183109],
    [187677, 187705],
    [203502, 203530],
    [219482, 219510],
    [235509, 235537], #20
    [275603, 275633],
    [277908, 277936],
    [301722, 301753],
    [322041, 322070],
    [322422, 322450],
    [339137, 339166],
    [393906, 393935],
    [395396, 395424],
    [396716, 396746],
    [412940, 412969], #30
    [413358, 413386], 
    [430261, 430289],
    [444404, 444433],
    [453962, 453990],
    [456065, 456095],
    [456863, 456892],
    [457187, 457215],
    [457390, 457419],
    [470756, 470783],
    [474540, 474568], #40
    [502372, 502400],
    [564199, 564227]
]

In [ ]:
manual_width = np.array(manual_width)
manual_width.shape

In [ ]:
width_df = pd.DataFrame(manual_width, columns=['start', 'end'])
width_df['samples'] = width_df['end'] - width_df['start']
width_df['duration'] = width_df['samples']/1000

In [ ]:
width_df[['samples','duration']].describe()

De los defectos encontrados, se puede aprecia que tienen una duración media de 28.6 muestras, con una desviación estándar de menos de 1 punto. Lo que dice que tienen una duración menor a los 0.03 segundos.
En general, los defectos tienen un ancho fijo de 0.03 segundos, dado que no hay una gran disperción en los datos.

### **tiempo de vida**

Para los tiempos carácteristicos por defecto, se toma el indice del peak y luego a mitad de la curva descendente se toma el indice. El doble de la cantidad de muestras es representativo del tau del defecto. Notar que este valor tiene que ser menor a la duración total (cerca de 0.03s) de los defectos.

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [ ]:
defectos[0].shape

In [ ]:
indices_tau = []
for elem in defectos:
    x = np.max(np.abs(elem['Gradient Phase']))
    indice1 = elem[np.abs(elem['Gradient Phase']) == x].index[0]

    y = x/2
    subset = elem[100:]
    z = find_nearest(np.abs(subset['Gradient Phase']), y)

    indice2 = subset[np.abs(subset['Gradient Phase']) == z].index[0]

    indices_tau.append([indice1, indice2])



In [ ]:
tau_df = pd.DataFrame(indices_tau, columns=['peak_idx', 'median_idx'])
tau_df['tau_samples'] = (tau_df['median_idx'] - tau_df['peak_idx'])*2
tau_df['tau'] = tau_df['tau_samples']/1000
tau_df.head()

In [ ]:
tau_df[['tau_samples', 'tau']].describe()

Se puede ver que los defectos tienen, en promedio, un tiempo de vida de 0.012 segundos, con una desviasión estandar de menos de $10^3$ segundos. Es decir, tienen un tiempo de vida de 12 muestras.

### **tiempo de aparición**

In [ ]:
width_df['difference'] = width_df['start'].shift(-1) - width_df['end']
width_df['amp_peak'] = df['Gradient Phase'][intersection].values
width_df.head()


In [ ]:
display(width_df[['difference', 'amp_peak']].describe())

notar que no sirven los indicadores normales de dispersión, dado que la desviación estandar es muy grande, esta es mayor al promedio, por lo que no es un buen indicador para ver como se distribuyen los datos. Se necesitan indicadores de dispersión robustos. En primera instancia, se revisaran histogramas para entender la distribución y luego ver otras opciones.

In [ ]:
fig = go.Figure(data=[go.Histogram(x=width_df['difference'])])
fig.update_layout(title='Histogram of Difference', xaxis_title='Difference', yaxis_title='Count')
fig.show()


#### **distribución de tiempos de aparición**

In [ ]:
appearance_time = width_df['difference'].values
appearance_time = appearance_time[~np.isnan(appearance_time)]

In [ ]:
f = Fitter(appearance_time,
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm",
                          "chi2"])
f.fit()
f.summary()

In [ ]:
f.get_best()

In [ ]:
forma, loc, escala = stats.chi2.fit(appearance_time)
estadisticos_ks, p_valor_ks = stats.kstest(appearance_time, 'gamma', args=(forma, loc, escala))

nivel_significancia = 0.05

if p_valor_ks < nivel_significancia:
    print("Se rechaza la hipótesis nula. Datos no siguen una distribución gamma")
else:
    print("No hay evidencia para rechazar hipotesis nula. Datos pueden una distribución gamma")

Con esto, se asume que los datos no siguen la distribución gamma que se encontró. Sin embargo, el obtener más datos es primordial para poder hacer una afirmación más certera. De igual manera, se puede utilizar la escala encontrada para hacer la partición de las ventanas de la señal.

In [ ]:

# Parameters obtained from f.get_best()
shape = 0.6535
scale = 13328.37838

x = np.linspace(0, np.max(appearance_time), 100)
# probability density function
y = stats.gamma.pdf(x, a=shape, scale=scale)

fig = go.Figure()
fig.add_trace(go.Histogram(x=appearance_time, histnorm='probability density', bingroup=30, name='Appearance Time Histogram'))
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Gamma Distribution'))
fig.update_layout(title='Gamma Distribution Fit', xaxis_title='Appearance Time', yaxis_title='Probability Density')
fig.show()

#histograma en matplotlib
plt.figure(figsize=(15, 10))
plt.hist(appearance_time, bins=30, density=True, alpha=0.5, label='Appearance Time Histogram')
plt.plot(x, y, 'r-', label='Gamma Distribution')
plt.xlabel('Appearance Time')
plt.ylabel('Probability Density')
plt.legend()
plt.show()


In [ ]:
fig = go.Figure(data=[go.Histogram(x=np.abs(width_df['amp_peak']))])
fig.update_layout(title='Histograma de peaks de amplitud en gradiente de fase', xaxis_title='GF', yaxis_title='Count')
fig.show()

asumiendo los peaks como una señal cualquiera. Se ve si se puede extraer algún tipo de información, como frecuencias presentes en la posible señal.

In [ ]:
plt.plot(df['Gradient Phase'][intersection], 'o-')
plt.title('Gradiente de fase de peaks')
plt.show()


In [ ]:

fft_result = fft(np.abs(df['Gradient Phase'][intersection].values))

N = len(df['Gradient Phase'][intersection])
sampling_rate = 1
normalize =  N/2.0

freq_axis = fftfreq(N, d=1.0/sampling_rate)
norm_amplitude = np.abs(sig_fft)/normalize

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = freq_axis, y = norm_amplitude, mode = 'lines'))
fig.update_layout(title = 'Espectro peaks', xaxis_title = 'Frecuencia [Hz]', yaxis_title = 'Amplitud')
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')
fig.show()

el espectro no indica mucho. Se ve un punto en los 0.5Hz que corresponde a un periodo de 2 segundos, implicando que, hay una posible periodicidad cada 2000 muestras. Esto es consistente solo con el 25% de los peaks.
Por otro lado, viendo los estadisticos, se puede ver que hasta la mitad de lo peaks, estos se presentan entre los 2 y 10 segundos. Por lo que se acontinuación se crean ondas sinusoidales con las frecuencias que se encuentran a los 25%, 50% y 75% para ver como esta curva se ajusta a los peaks.


In [ ]:

t = np.linspace(intersection[0], intersection[-1], len(intersection), endpoint=False)

# Generate the sine waves
frequencies = [2008, 9289, 16784]#, 61799]
amp = [-0.306064, -0.275678, 0.262373]#, 0.297188]

sine_waves = [amp[i] * np.sin(2 * np.pi * frequencies[i] * t) for i in range(len(frequencies))]

# Sum the sine waves
combined_signal = np.sum(sine_waves, axis=0)

# Plot the combined signal
plt.plot(t, combined_signal)
plt.plot(intersection, df['Gradient Phase'][intersection], 'o')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Combined Sine Waves')
plt.show()


## **correlación**


### **con el resto de la señal**

In [ ]:
#tomo 100 pts antes y después del defecto nº 1 y saco la correlación con el resto de la señal

defecto_1 = defectos[0]

plt.figure(figsize=(7,5))
plt.plot(np.abs(defecto_1['Gradient Phase']))
plt.title('Defecto nº 1')
plt.show()

In [ ]:
#señal a la que le vamos a hacer correlación

signal_grad = np.abs(df['Gradient Phase'])
plt.figure(figsize=(15,5))
plt.plot(signal_grad)
plt.title('Gradiente de fase, señal completa')
plt.show()

In [ ]:
correlation = signal.correlate(np.abs(defecto_1['Gradient Phase']), signal_grad, mode = 'full')

fig = go.Figure()
fig.add_trace(go.Scatter(y = correlation, name = 'Correlación'))
fig.add_trace(go.Scatter(y = signal_grad, opacity=0.5, name = 'Gradiente de fase'))
fig.update_layout(title = 'Correlación defecto nº 1 con señal completa')
fig.show()



no se puede ver de manera clara que los puntos de más alta correlación sean los peaks de la señal. Se descarta para el análisis.

## **entre defectos**

In [ ]:
#amplitud máxima encontrada
max_amplitude = np.max([np.max(np.abs(defecto['Gradient Phase'])) for defecto in defectos])

#normalización de los defectos.
for defecto in defectos:
    defecto['Gradient Phase Norm'] = defecto['Gradient Phase'] / max_amplitude

In [ ]:
correlations = []
for i in range(len(defectos)):
    current_corr = []
    for j in range(len(defectos)):
        current_corr.append(signal.correlate(defectos[i]['Gradient Phase Norm'], defectos[j]['Gradient Phase Norm'], mode = 'same'))
    correlations.append(current_corr)

In [ ]:
for i, array in enumerate(correlations):
    fig = go.Figure()
    for elem in array:
      fig.add_trace(go.Scatter(x = t, y = elem, mode= 'lines'))
      fig.update_layout(title = f'Correlación cruzada defecto {i}')
    fig.show()